<a href="https://colab.research.google.com/github/cannin/gsoc_2023_pytorch_pathway_commons/blob/main/Building_GNNmodel_with_PyG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch-geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data

In [3]:
cBio = pd.read_csv('/content/cBio (1).csv')

In [4]:
cBio.head()

,Sample Identifier,Overall Survival (Months),NUDT18,SHC3,NCRNA00112,FAM49B,FMO6P,BAT2L2,SIRT1,MSLNL,...,SCARNA6,SNORD77,TLR10,GKN2,PCDHGB4,ZMAT5,AXIN2,LONRF1,KLHL10,C12orf36
0,TCGA-OR-A5J1-01,44.547457,85.0144,97.5024,0.0,671.950,0.0,2428.43,462.056,0.0,...,0.0000,0.0,12.0077,0.0,17.2911,913.535,547.5500,363.112,1.9212,0.0
1,TCGA-OR-A5J2-01,55.133642,165.8840,92.6050,0.0,812.911,0.0,5097.30,669.977,0.0,...,0.4026,0.0,0.8053,0.0,268.9890,303.978,1793.3200,592.672,0.0000,0.0
2,TCGA-OR-A5J3-01,68.744452,309.2880,10.0726,0.0,482.299,0.0,1615.76,514.294,0.0,...,0.5925,0.0,0.0000,0.0,689.4860,324.592,522.5890,456.821,0.0000,0.0
3,TCGA-OR-A5J5-01,11.999868,178.9310,49.5740,0.0,466.305,0.0,1202.17,591.789,0.0,...,0.0000,0.0,6.9713,0.0,54.2215,790.550,337.7230,484.895,1.5492,0.0
4,TCGA-OR-A5J6-01,88.864780,124.0660,5.5886,0.0,1074.680,0.0,1424.53,805.312,0.0,...,1.6766,0.0,1.1177,0.0,8.9417,291.550,9.5006,628.155,1.1177,0.0


In [5]:
path = "/content/drive/MyDrive/gsoc data/PathwayCommons12.reactome.hgnc.sif.gz"  


#specify compression type because the file is compressed
df = pd.read_csv(path, sep="\t", compression="gzip", header=None, names=["Source", "InteractionType", "Target"])

In [6]:
df.head()

,Source,InteractionType,Target
0,A1CF,in-complex-with,APOBEC1
1,A1CF,in-complex-with,APOBEC2
2,A1CF,in-complex-with,APOBEC3A
3,A1CF,in-complex-with,APOBEC3B
4,A1CF,in-complex-with,APOBEC3C


In [7]:
reduced_df = df.drop(columns='InteractionType')

In [24]:
reduced_df.head()

,Source,Target
0,A1CF,APOBEC1
1,A1CF,APOBEC2
2,A1CF,APOBEC3A
3,A1CF,APOBEC3B
4,A1CF,APOBEC3C


In [25]:
reduced_df.shape

(353636, 2)

In [26]:
cBio.shape

(78, 20513)

In [8]:
source_nodes = reduced_df['Source'].tolist()
target_nodes = reduced_df['Target'].tolist()

# Create a set of all unique nodes
all_nodes = set(source_nodes + target_nodes)

# Create a dictionary to map each unique node to a unique index
node_to_index = {node: index for index, node in enumerate(all_nodes)}

# Map the source and target nodes to their corresponding indices
source_indices = [node_to_index[node] for node in source_nodes]
target_indices = [node_to_index[node] for node in target_nodes]

# Convert the source and target indices to a PyTorch tensor
edge_index = torch.tensor([source_indices, target_indices], dtype=torch.long)

# Create the graph data object
graph_data = Data(edge_index=edge_index)

In [9]:
graph_data.num_nodes

/usr/local/lib/python3.10/dist-packages/torch_geometric/data/storage.py:304: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_index'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


12324

In [10]:
graph_data.num_edges

353636

In [11]:
graph_data.edge_index

tensor([[11563, 11563, 11563,  ..., 11429, 11429, 11429],
        [ 5198,  7994,  2941,  ...,  5554,  3090,  7015]])

In [19]:
from torch.utils.data import random_split


# Extract the features and overall survival columns from the DataFrame
features = cBio.drop(["Sample Identifier","Overall Survival (Months)"], axis=1).values
survival = cBio["Overall Survival (Months)"].values

# Convert the features and survival data to PyTorch tensors
x = torch.tensor(features, dtype=torch.float)
y = torch.tensor(survival, dtype=torch.float).view(-1, 1)  # Reshape to (num_samples, 1)

# Create the PyTorch Geometric Data object
data = Data(x=x, y=y)

In [35]:
x

tensor([[8.5014e+01, 9.7502e+01, 0.0000e+00,  ..., 3.6311e+02, 1.9212e+00,
         0.0000e+00],
        [1.6588e+02, 9.2605e+01, 0.0000e+00,  ..., 5.9267e+02, 0.0000e+00,
         0.0000e+00],
        [3.0929e+02, 1.0073e+01, 0.0000e+00,  ..., 4.5682e+02, 0.0000e+00,
         0.0000e+00],
        ...,
        [2.3205e+02, 6.9790e-01, 0.0000e+00,  ..., 2.3938e+02, 1.0468e+00,
         0.0000e+00],
        [3.0375e+02, 1.9294e+02, 0.0000e+00,  ..., 6.7255e+02, 5.5130e-01,
         0.0000e+00],
        [1.5998e+02, 5.0521e+01, 0.0000e+00,  ..., 1.2109e+02, 4.0100e-01,
         0.0000e+00]])

In [36]:
x.shape

torch.Size([78, 20511])

In [37]:
y.shape

torch.Size([78, 1])

In [23]:
data.num_nodes

78

In [53]:
# Step 1: Extract Gene Names
cbio_gene_names = cBio.columns[2:].tolist()
path_gene_names = pd.unique(reduced_df[["Source", "Target"]].values.ravel())

# Step 2: Find Overlapping Gene Names
overlapping_genes = list(set(cbio_gene_names) & set(path_gene_names))


In [54]:
# Step 3: Filter and Transpose cBio Data
filtered_cBio_data = cBio[['Sample Identifier', 'Overall Survival (Months)'] + overlapping_genes]
filtered_cBio_data = filtered_cBio_data.rename(columns={'Sample Identifier': 'sample_id'})
filtered_cBio_data = filtered_cBio_data.set_index('sample_id').transpose()


In [55]:
# Step 4: Process Pathway Commons Data for Graph Connectivity
path_gene_names = pd.Index(overlapping_genes)
path_node_indices = path_gene_names.get_indexer(reduced_df[['Source', 'Target']].values.ravel())
edge_index = torch.tensor(path_node_indices.reshape(-1, 2).T, dtype=torch.long)

In [57]:
print("Filtered cBio Data:")
filtered_cBio_data.head()


Filtered cBio Data:


sample_id,TCGA-OR-A5J1-01,TCGA-OR-A5J2-01,TCGA-OR-A5J3-01,TCGA-OR-A5J5-01,TCGA-OR-A5J6-01,TCGA-OR-A5J7-01,TCGA-OR-A5J8-01,TCGA-OR-A5J9-01,TCGA-OR-A5JA-01,TCGA-OR-A5JB-01,...,TCGA-OR-A5LR-01,TCGA-OR-A5LS-01,TCGA-OR-A5LT-01,TCGA-OU-A5PI-01,TCGA-P6-A5OF-01,TCGA-PA-A5YG-01,TCGA-PK-A5H8-01,TCGA-PK-A5H9-01,TCGA-PK-A5HA-01,TCGA-PK-A5HB-01
Overall Survival (Months),44.547457,55.133642,68.744452,11.999868,88.86478,16.109413,19.035408,44.448828,30.311997,18.11487,...,28.043528,36.032482,18.049117,38.498208,6.805405,24.854522,119.111023,20.251833,39.484499,42.509123
MIP,3.362200,4.026300,12.442600,11.618900,1.67660,5.214600,0.000000,19.378900,20.084600,0.44560,...,2.510500,14.907700,0.850300,19.384300,7.039500,3.698600,2.498400,1.744700,0.551300,0.801900
ADAM20,2.881800,1.610500,4.740000,13.942700,0.00000,0.000000,2.072100,2.719800,1.057100,2.22790,...,0.836800,3.388100,1.700700,2.280500,1.955400,0.462300,4.996900,1.046800,0.551300,0.401000
TRPC7,0.000000,0.000000,2.370000,0.000000,0.00000,0.401100,0.000000,0.000000,0.528500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
PSAT1,4769.450000,4129.380000,2913.940000,1724.240000,298.98800,4510.230000,2600.500000,4684.250000,1186.580000,1416.95000,...,685.356000,3763.510000,12324.000000,3050.740000,2403.600000,3684.700000,747.658000,6636.660000,504.961000,2505.610000


In [59]:
filtered_cBio_data.shape

(9615, 78)

In [58]:
print("\nEdge Index:")
print(edge_index)


Edge Index:
tensor([[9021, 9021, 9021,  ..., 8922, 8922, 8922],
        [4088, 6236, 2323,  ...,   -1, 2439, 5494]])


In [60]:
x = torch.tensor(filtered_cBio_data.iloc[1:].values[1:], dtype=torch.float32)
y = torch.tensor(filtered_cBio_data.iloc[0, 1:], dtype=torch.float32)

In [61]:
x = x.transpose(0, 1)

In [62]:
data = Data(x=x, y=y, edge_index=edge_index, num_node_features=x.size(1))

In [63]:
print(data)


Data(x=[78, 9613], edge_index=[2, 353636], y=[77], num_node_features=9613)


In [66]:
from torch.utils.data import DataLoader

from torch.utils.data import random_split, DataLoader

# Define the train and test dataset sizes
train_size = int(len(data) * 0.8)  # 80% for training
test_size = len(data) - train_size  # Remaining 20% for testing

# Split the dataset into train and test sets
train_data, test_data = random_split(data, [train_size, test_size])

# Create data loaders for batching
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


In [67]:
print(train_data)
print(test_data)

In [77]:
from torch.utils.data import Subset

# Define the size of the train and test datasets
train_size = int(0.8 * len(data))
test_size = len(data) - train_size

# Define the indices for train and test sets
train_indices = range(train_size)
test_indices = range(train_size, len(data))

# Create the train and test subsets
train_data = Subset(data, train_indices)
test_data = Subset(data, test_indices)

# Print samples from the train dataset
for sample in train_data:
    print(sample)

# Print samples from the test dataset
for sample in test_data:
    print(sample)


KeyError: ignored